**Wed, Oct 11th**

This notebook explores SAM foundational model from FAIR for creating masks from satellite imagery. The expected outcome is to de-risk:

1. Creating a valuable dataset for understanding changes in a location.
2. Generating masks / detecting objects from images 

In [2]:
import torch
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

ModuleNotFoundError: No module named 'transformers'

In [ ]:


device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[450, 600]]]  # 2D location of a window in the image

inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)
outputs = model(**inputs)

masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)
scores = outputs.iou_scores